<a href="https://colab.research.google.com/github/GoTudering/Deep_Learning_Study/blob/main/Tree_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
'''
정형 데이터와 비정형 데이터

정형 데이터: 어떤 구조로 되어있어 CSV나 데이터베이스 엑셀에 저장하기 쉬움
비정형 데이터: 정형 데이터의 반대. 덱스트 데이터, 사진, 디지털 음악 등이 있음

머신러닝은 정형 데이터에 잘 맞음 비정형 데이터는 너무 복잡하여 신경망 학습이 필요함
정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습ensemble learning임
이 알고리즘은 대부분 결정 트리를 기반으로 만들어져있음

랜덤 포레스트는 앙상블 학습의 대표주자 중 하나로 앙상블 학습을 적용할 때 가장 먼저 랜덤 포레스트를 시도해보는 것이 좋음
랜덤 포레스트는 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듦. 그 방법은 훈련 데이터에서 랜덤하게 샘플을 추출하여(중복가능) 훈련데이터를 만듦
이를 부트스트램 샘플bootstrap sample이라고 함(훈련 세트 크기 만큼 추출하기에 그 크기는 같음)
또한 각 노드를 분할할 때 전체 특성 중 일부 특성을 무작위로 고른 다음 최선의 분할을 찾음
분류모델인 RandomForestClassifier은 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택하고 회귀 모델인 RandomForestRegressor는 전체 특성을 사용함
사이킷런의 랜덤 포레스트는 기본적으로 100개의 결정 트리를 훈련함
분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고 회귀일 때는 단순히 각 트리의 예측을 평균함
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine[['class']].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [13]:
#cross_validate()함수를 사용해 교차 검증
#RandomForestClassifier은 기본적으로 100개의 결정 트리를 사용하므로 n_jobs매개변수를 -1로 지정하여 모든 CPU코어를 사용하는 것이 좋음
#cross_validate()함수의 n_jobs 매개변수도 -1로 지정하여 최대한 병렬로 교차 검증을 수행
#return_train_score 매개변수를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환함(기본값은 False임)
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(rf,train_input,train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [14]:
#랜덤 포레스트는 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공함
#criterion,max_depth,max_features,min_samples_split,min_impurity_decrease,min_samples_leaf
#또한 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산함
rf.fit(train_input,train_target)
print(rf.feature_importances_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[0.23167441 0.50039841 0.26792718]


In [15]:
#RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있음
#부트스트램 샘플에 포함되지 않고 남은 샘플이 있음 이를 OOB(out of bag)샘플 이라고함
#이 샘플로 평가가능
#이 점수를 얻으려면 RandomForestClassifier클래스의 oob_score 매개변수를 True로 지정해야함(기본적으로 False)
#이렇게 하면 랜덤 포레스트는 각 결정 트리의 OOB점수를 평균하여 출력함
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)
#OOB점수를 사용하면 교차 검증을 대신할 수 있으서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있음

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


0.8934000384837406


In [16]:
#엑스트라 트리 Extra Trees
#랜덤 포레스트ㅘ 매우 비슷. 기본적으로 100개의 결정트리를 훈련하고 결정 트리가 제공하는 대부분의 매개변수를 지원함
#전체 특성 중 일부 특성을 랜덤하게 선택하여 사용
#부트스트랩 샘플을 사용하지 않음 결정 트리를 만들 때 전체 훈련 세트를 사용
#대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할함
#엑스트라 트리가 사용하는 결정 트리가 바로 DecisionTreeClassifier 의 splitter 매개변수가 'random'인 결정트리임
#하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기에 과대적합을 먹고 검증 세트의 점수를 높임
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [17]:
#특성 중요도도 제공
et.fit(train_input,train_target)
print(et.feature_importances_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[0.20183568 0.52242907 0.27573525]


In [22]:
#그레이디언트 부스팅 gradient boosting은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
#GradientBoostingClassifier는 기본적으로 깊이가 3인 결정 트리를 100개 사용함
#깊이가 얕아 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음
#경사 하강법을 사용하여 트리를 앙상블에 추가함
#분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용함
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [23]:
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [24]:
#그레이디언트 부스팅도 특성 중요도를 제공함
gb.fit(train_input,train_target)
print(gb.feature_importances_)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.15872278 0.68010884 0.16116839]


In [26]:
#트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample 매개변수가 있음
#기본값은 1 이보다 작으면 훈련 세트의 일부를 사용함
#이는 마치 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 확률적 경사 하강법이나 미니배치 경사 하강법과 비슷

#일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있지만 순서대로 트리를 추가하기 때문에 훈련속도도 느림(n_jobs매개변수가 없음)

#그레이디언트 부스팅의 속도와 성능을 더욱 개선한 것이 히스토그램 기반 그레이디언트 부스팅임

#히스토그램 기반 그레이디언트 부스팅 Histogram-based Gradient Boosting 은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘임
#입력 특성을 256개의 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
#구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용하기에 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

0.9321723946453317 0.8801241948619236


In [27]:
#히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importance()함수를 사용
#특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
#n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정(기본값은 5)
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
result=permutation_importance(hgb,train_input,train_target,
                              n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.08876275 0.23438522 0.08027708]


In [28]:
result=permutation_importance(hgb,test_input,test_target,
                              n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [29]:
hgb.score(test_input,test_target)

0.8723076923076923

In [30]:
#XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist',random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8824322471423747 0.8726214185237284


In [31]:
#LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
